In [76]:
# importar llibreries
import pandas as pd

from repositories import csvTRNSYS, csvTRNSYS2
from statistics import mean
from interactors import estimationPV
from sqlalchemy import text
from  SQL_statements import database_connection
from utils import meanCalculation, mean_calculation2, meanCalculation2
from utils import energyConsumption

In [77]:
# arguments

"""
Distribución de instalaciones base para el cálculo de medidas activas.
La distribución siempre será igual a la de analisysTRNSYS, de todas formas se ha dejado abierto por si fuera necesario modificarlo
"""

grado_ht = {'joule':0.32,'BC':0.31,'caldera_GN':0.27,'butano':0.06,'nada':0.04}
grado_cl = {'split':0.1,'BC':0.43,'multi_split':0.04,'nada':0.43}
grado_dhw ={'caldera_GN':0.86,'butano':0.14,'BC':0}

EER_nrt = 2
COP_nrt = 2
COP_DHW_nrt = 1.7
eta_cal_nrt = 0.7
EER_rt = 5
COP_rt = 4.6
COP_DHW_rt = 3.17
eta_cal_rt = 0.92

''' 
name = 'Caracas1'
n_viv = 3
ref_cat = '1502002DD7810B'
'''

n_viv = 1
name = 'FE35'
ref_cat = '1701521DD7810B'

In [78]:
# LLegir demandes de TRNSYS
typeStep = csvTRNSYS2.HourlyMulti() 
output = csvTRNSYS2.readTRNSYS(typeStep, n_viv, name, ref_cat)
output_reader, P_ht, P_cl, P_lig, P_dev, P_dhw = output.start()
P_ht



{'BC0_0': {'Dwelling_1': [0.0,
   0.013943688146161613,
   0.01577031613303688,
   0.015819811390408996,
   0.015262582061859575,
   0.003969653372767092,
   0.011340362627703519,
   0.020114567865391297,
   0.019796338209640354,
   0.019423308729197553,
   0.006159056495354009,
   0.02132221598859055,
   0.018153159352215186,
   0.01990158574761529,
   0.020288732079677383,
   0.020415484009152017,
   0.021187196336462624,
   0.008494705026428389,
   0.022538150835669398,
   0.011442719594427922,
   0.019686792225199472,
   0.023079678982251193,
   0.004740259417732996,
   0.02649759524514946,
   0.025620596797737726,
   0.028044397317894484,
   0.029340261868487767,
   0.03308944416212284,
   0.0336445562981299,
   0.020026010035809286,
   0.02269394244174368,
   0.015307983689162696,
   0.037593452154285945,
   0.016632246840402064,
   0.044663455064613694,
   0.03536051282192552,
   0.03287900831318343,
   0.03360468190453456,
   0.034275662684190085,
   0.03298638995618194,
   0.0

In [79]:
project = meanCalculation2.ARV_dwellings()
output = meanCalculation2.calculo(project,  P_ht, P_cl, P_lig, P_dev, P_dhw,name)
P_ht_mean, P_cl_mean, P_lig_mean, P_dev_mean, P_dhw_mean = output.start()
P_ht_mean

{'BC0_0': 126.42762452618094,
 'BC0_180': 126.44455845010127,
 'BC0_270': 129.46856923583837,
 'BC0_90': 118.68447848515403,
 'P111_0_W8_R10': 71.82033099957293,
 'P111_0_W8_R12': 70.31584885156512,
 'P111_0_W8_R14': 69.16042949194612,
 'P111_0_W8_R6': 76.77939107655891,
 'P111_0_W8_R8': 73.86221400632598,
 'P111_180_W8_R10': 71.73461675386389,
 'P111_180_W8_R12': 70.2260937856505,
 'P111_180_W8_R14': 69.06769395211033,
 'P111_180_W8_R6': 76.70700352839218,
 'P111_180_W8_R8': 73.77951777707071,
 'P111_270_W8_R10': 75.14820091521827,
 'P111_270_W8_R12': 73.5628081676869,
 'P111_270_W8_R14': 72.34429184308289,
 'P111_270_W8_R6': 80.36409484946616,
 'P111_270_W8_R8': 77.29400416511044,
 'P111_90_W8_R10': 63.25676814449211,
 'P111_90_W8_R12': 61.794811133026606,
 'P111_90_W8_R14': 60.67956633458144,
 'P111_90_W8_R6': 68.08118561727844,
 'P111_90_W8_R8': 65.23967855407183,
 'P112_0_W8_R10': 73.36912837986688,
 'P112_180_W8_R10': 73.28357758141733,
 'P112_270_W8_R10': 76.77611060937674,
 'P1

In [84]:
def process_dweelings(P_ht, P_cl, P_lig, P_dev, P_dhw, name):
    kwargs = {
        'P_ht': P_ht,
        'P_cl': P_cl,
        'P_lig': P_lig,
        'P_dev': P_dev,
        'P_dhw': P_dhw
    }
    output = {}
    for key, vector in kwargs.items():
        num_rows = len(pd.DataFrame(vector))
        result_dict = {}
        for column in pd.DataFrame(vector):
            result_list = [sum(x) / num_rows for x in zip(*pd.DataFrame(vector)[column])]
            result_dict[column] = [result_list]
        new_df = pd.DataFrame(result_dict)
        if name == 'Caracas1':
            simulations = set(new_df.columns.str.rsplit("_", n=1).str[0])
        else: 
            simulations = new_df.columns
        summed_results = {}
        for sim in simulations:
            relevant_columns = [col for col in new_df.columns if col.startswith(sim)]
            summed_results[sim] = []
            for index, row in new_df.iterrows():
                sum_list = [sum(elements) for elements in zip(*[row[col] for col in relevant_columns])]
                summed_results[sim].append(sum_list)
        output[key] = summed_results
        
    return output['P_ht'], output['P_cl'], output['P_lig'], output['P_dev'], output['P_dhw']

In [86]:
P_ht_hourly, P_cl_hourly, P_lig_hourly, P_dev_hourly, P_dhw_hourly = process_dweelings(P_ht, P_cl, P_lig, P_dev, P_dhw, name)

In [82]:
pd.DataFrame(P_ht_hourly)

,P121_90_W6,P211_270_W8,P121_0_W8,P221_270_W8,P221_0_W6,P111_270_W8,P222_0_W8,P112_90_W8,P122_180_W8,P221_90_W8,...,P122_90_W8,P111_90_W8,P221_90_W6,P221_0_W8,P211_90_W8,P112_270_W8,P112_180_W8,P122_90_W6,P212_90_W8,P222_0_W6
0,"[0.0, 0.013936745143606221, 0.0156451162593235...","[0.0, 0.0558009727984616, 0.061749682473558984...","[0.0, 0.01393674513961679, 0.01564511538435103...","[0.0, 0.01117183112702784, 0.01236465035777076...","[0.0, 0.011171831131033746, 0.0123646512362125...","[0.0, 0.0696837268782084, 0.07822564227174714,...","[0.0, 0.011171830979057095, 0.0123646435770480...","[0.0, 0.013936745098738369, 0.0156451069329519...","[0.0, 0.01393674499250715, 0.01564510864045444...","[0.0, 0.01117183112702784, 0.01236465035777076...",...,"[0.0, 0.01393674499250715, 0.01564510864045444...","[0.0, 0.0696837268782084, 0.07822564227174714,...","[0.0, 0.011171831131033746, 0.0123646512362125...","[0.0, 0.01117183112702784, 0.01236465035777076...","[0.0, 0.0558009727984616, 0.061749682473558984...","[0.0, 0.013936745098738369, 0.0156451069329519...","[0.0, 0.013936745098738369, 0.0156451069329519...","[0.0, 0.013936744996496593, 0.0156451095154269...","[0.0, 0.011160194281155219, 0.0123499148518384...","[0.0, 0.01117183098306302, 0.01236464445548981..."


In [87]:
P_ht_hourly['BC0_0'][0][2]

0.01577031613303688

In [14]:
P_ht_hourly['P221_180_W6_R14'][0][2]

0.03441140577610222

In [15]:
Msplit = [1, 1, 1, 1, 0, 0, 0, 0]
PV = [1, 1, 0, 0, 1, 1, 0, 0]
BC = [1, 0, 1, 0, 1, 0, 1, 0]

In [48]:
P_ht_ele_rt = {}
P_cl_ele_rt = {}
P_dhw_ele_rt = {}
P_ht_ele_nrt = {}
P_cl_ele_nrt = {}
P_lig_ele = {}
P_dev_ele = {}

for key, values in P_ht_hourly.items():
    P_ht_ele_rt[key] = [[x / COP_rt for x in lst] for lst in values]
    P_cl_ele_rt[key] = [[x / EER_rt for x in lst] for lst in values]
    P_dhw_ele_rt[key] = [[x / COP_DHW_rt for x in lst] for lst in P_dhw_hourly[key]]
    P_ht_ele_nrt[key] = [[x * (grado_ht['joule'] + (grado_ht['BC']) / COP_nrt) for x in lst] for lst in values]
    P_cl_ele_nrt[key] = [[x / EER_nrt for x in lst] for lst in values]
    P_lig_ele[key] = P_lig_hourly[key]
    P_dev_ele[key] = P_dev_hourly[key]


In [44]:
P_ht_ele_rt['BC0_0'][0][2]

0.008139116310176279

In [46]:
P_ht_hourly['BC0_0'][0][2]/COP_rt

0.008139116310176279

In [24]:
pd.DataFrame(P_ht_hourly)

,P211_180_W12_R6,P121_0_W8_R12,P211_270_W10_R12,P211_180_W6_R8,BC0_270,P112_180_W8_R14,P221_270_W10_R6,P221_90_W6_R14,P121_180_W12_R8,P111_0_W8_R6,...,P112_270_W8_R10,P111_270_W10_R14,P221_180_W12_R10,P121_180_W10_R12,P221_270_W10_R10,P121_90_W12_R6,P221_90_W12_R12,P211_180_W8_R12,P121_90_W12_R14,P211_270_W8_R14
0,"[0.0, 0.053973692948462465, 0.0343994893564898...","[0.0, 0.05579984709763176, 0.03657570864137924...","[0.0, 0.053973893452550964, 0.0344019002842902...","[0.0, 0.05397499333741011, 0.0344157949001009,...","[0.0, 0.05588335174349485, 0.03743993502681088...","[0.0, 0.055799975684617975, 0.0365780670562585...","[0.0, 0.053973675189128774, 0.0343991684403993...","[0.0, 0.05397466546573976, 0.03441140577610222...","[0.0, 0.055799272483118646, 0.0365676226941249...","[0.0, 0.05580020340754439, 0.03658051979787235...",...,"[0.0, 0.05579997868564705, 0.03657819802691448...","[0.0, 0.05579976556183583, 0.03657407042674228...","[0.0, 0.05397345390229896, 0.03439554206627172...","[0.0, 0.05579948348557365, 0.03657079380396463...","[0.0, 0.05397366525725904, 0.03439878784037702...","[0.0, 0.055799281018594736, 0.0365679336241415...","[0.0, 0.053973453678237596, 0.0343955251321359...","[0.0, 0.053974290950996345, 0.0344071543078237...","[0.0, 0.05579927072350971, 0.03656752731839316...","[0.0, 0.05397429032299869, 0.03440712118315096..."


In [49]:
 output_scenarios = {}
 for i in P_ht_mean:
            output_scenarios.setdefault(i, {})
            for j in range(0, len(Msplit)):
                key = []
                if Msplit[j] == 1:
                    key.append('Split_rf+')
                else:
                    key.append('Split_nrf+')
                if PV[j] == 1:
                    key.append('PV_yes+')
                else:
                    key.append('PV_no+')
                if BC[j] == 1:
                    key.append('BC_yes')
                else:
                    key.append('BC_no')
                output_scenarios[i].setdefault(("".join(key)),{'GN':0,'Ele':0,'Butano':0}) # Se inicializan los vectores energéticos



In [71]:
PV_estimation = {}
total_electricity = {}

for i in output_scenarios:
    PV_estimation.setdefault(i, {})
    total_electricity.setdefault(i,{})
    for j in output_scenarios[i]:
        P_ele_total = [0] * 8760  # Generación de una lista con 8760 ceros
        if 'Split_rf' in j:
            output_scenarios[i][j]['Ele'] = output_scenarios[i][j]['Ele'] + (P_cl_mean[i]) / EER_rt + P_ht_mean[i] / COP_rt
        if 'Split_nrf' in j:
            output_scenarios[i][j]['Ele'] = output_scenarios[i][j]['Ele'] + (P_cl_mean[i]) / EER_nrt + P_ht_mean[i] * (grado_ht['joule'] + (grado_ht['BC']) / COP_nrt)
            output_scenarios[i][j]['GN'] = output_scenarios[i][j]['GN'] + P_ht_mean[i] * ((grado_ht['caldera_GN'] + grado_ht['nada']) / eta_cal_nrt)
            output_scenarios[i][j]['Butano'] = output_scenarios[i][j]['Butano'] + P_ht_mean[i] * grado_ht['butano'] / eta_cal_nrt
        if 'BC_yes' in j:
            output_scenarios[i][j]['Ele'] = output_scenarios[i][j]['Ele'] + P_dhw_mean[i] / COP_DHW_rt
        if 'BC_no' in j:
            output_scenarios[i][j]['GN'] = output_scenarios[i][j]['GN'] + P_dhw_mean[i] * (grado_dhw['caldera_GN'] / eta_cal_nrt)
            output_scenarios[i][j]['Butano'] = output_scenarios[i][j]['Butano'] + P_dhw_mean[i] * (grado_dhw['butano'] / eta_cal_nrt)
        if 'PV_yes' in j: # Lógica de control para introducir la demanda en el código de PV
            if 'Split_rf' in j:
                P_ele_total = [x + y for x, y in zip(P_ele_total, P_cl_ele_rt[i][0])]
                P_ele_total = [x + y for x, y in zip(P_ele_total, P_ht_ele_rt[i][0])]
            if 'Split_nrf' in j:
                P_ele_total = [x + y for x, y in zip(P_ele_total, P_cl_ele_nrt[i][0])]  
                P_ele_total = [x + y for x, y in zip(P_ele_total, P_ht_ele_nrt[i][0])]
            if 'BC_yes' in j:
                P_ele_total = [x + y for x, y in zip(P_ele_total, P_dhw_ele_rt[i][0])] 
                
        total_electricity[i][j] = P_ele_total

In [73]:
total_electricity = pd.DataFrame(total_electricity)

In [63]:
pd.DataFrame(output_scenarios)

,P211_180_W12_R6,P121_0_W8_R12,P211_270_W10_R12,P211_180_W6_R8,BC0_270,P112_180_W8_R14,P221_270_W10_R6,P221_90_W6_R14,P121_180_W12_R8,P111_0_W8_R6,...,P112_270_W8_R10,P111_270_W10_R14,P221_180_W12_R10,P121_180_W10_R12,P221_270_W10_R10,P121_90_W12_R6,P221_90_W12_R12,P211_180_W8_R12,P121_90_W12_R14,P211_270_W8_R14
Split_rf+PV_yes+BC_yes,"{'GN': 0, 'Ele': 335.507094340509, 'Butano': 0}","{'GN': 0, 'Ele': 94.95906833415961, 'Butano': 0}","{'GN': 0, 'Ele': 112.55530917709635, 'Butano': 0}","{'GN': 0, 'Ele': 96.6151750138593, 'Butano': 0}","{'GN': 0, 'Ele': 166.59686781907135, 'Butano': 0}","{'GN': 0, 'Ele': 101.88614157415783, 'Butano': 0}","{'GN': 0, 'Ele': 112.84305900742598, 'Butano': 0}","{'GN': 0, 'Ele': 97.04547804034632, 'Butano': 0}","{'GN': 0, 'Ele': 100.43597595297427, 'Butano': 0}","{'GN': 0, 'Ele': 96.6887276691227, 'Butano': 0}",...,"{'GN': 0, 'Ele': 111.00860587548854, 'Butano': 0}","{'GN': 0, 'Ele': 110.06918407660942, 'Butano': 0}","{'GN': 0, 'Ele': 94.47179613255148, 'Butano': 0}","{'GN': 0, 'Ele': 100.02068923543595, 'Butano': 0}","{'GN': 0, 'Ele': 112.5558219448559, 'Butano': 0}","{'GN': 0, 'Ele': 103.10086660160358, 'Butano': 0}","{'GN': 0, 'Ele': 95.77460372847084, 'Butano': 0}","{'GN': 0, 'Ele': 95.22450124352494, 'Butano': 0}","{'GN': 0, 'Ele': 101.01638545932882, 'Butano': 0}","{'GN': 0, 'Ele': 112.48345269974548, 'Butano': 0}"
Split_rf+PV_yes+BC_no,"{'GN': 291.35615879134355, 'Ele': 21.048330124...","{'GN': 291.35615879134355, 'Ele': 20.148228647...","{'GN': 291.35615879134355, 'Ele': 37.744469490...","{'GN': 291.35615879134355, 'Ele': 21.804335327...","{'GN': 291.35615879134355, 'Ele': 91.786028132...","{'GN': 291.35615879134355, 'Ele': 27.075301887...","{'GN': 291.35615879134355, 'Ele': 38.032219320...","{'GN': 291.35615879134355, 'Ele': 22.234638353...","{'GN': 291.35615879134355, 'Ele': 25.625136266...","{'GN': 291.35615879134355, 'Ele': 21.877887982...",...,"{'GN': 291.35615879134355, 'Ele': 36.197766188...","{'GN': 291.35615879134355, 'Ele': 35.258344389...","{'GN': 291.35615879134355, 'Ele': 19.660956445...","{'GN': 291.35615879134355, 'Ele': 25.209849548...","{'GN': 291.35615879134355, 'Ele': 37.744982258...","{'GN': 291.35615879134355, 'Ele': 28.290026914...","{'GN': 291.35615879134355, 'Ele': 20.963764041...","{'GN': 291.35615879134355, 'Ele': 20.413661556...","{'GN': 291.35615879134355, 'Ele': 26.205545772...","{'GN': 291.35615879134355, 'Ele': 37.672613013..."
Split_rf+PV_no+BC_yes,"{'GN': 0, 'Ele': 95.85916981157399, 'Butano': 0}","{'GN': 0, 'Ele': 94.95906833415961, 'Butano': 0}","{'GN': 0, 'Ele': 112.55530917709635, 'Butano': 0}","{'GN': 0, 'Ele': 96.6151750138593, 'Butano': 0}","{'GN': 0, 'Ele': 166.59686781907135, 'Butano': 0}","{'GN': 0, 'Ele': 101.88614157415783, 'Butano': 0}","{'GN': 0, 'Ele': 112.84305900742598, 'Butano': 0}","{'GN': 0, 'Ele': 97.04547804034632, 'Butano': 0}","{'GN': 0, 'Ele': 100.43597595297427, 'Butano': 0}","{'GN': 0, 'Ele': 96.6887276691227, 'Butano': 0}",...,"{'GN': 0, 'Ele': 111.00860587548854, 'Butano': 0}","{'GN': 0, 'Ele': 110.06918407660942, 'Butano': 0}","{'GN': 0, 'Ele': 94.47179613255148, 'Butano': 0}","{'GN': 0, 'Ele': 100.02068923543595, 'Butano': 0}","{'GN': 0, 'Ele': 112.5558219448559, 'Butano': 0}","{'GN': 0, 'Ele': 103.10086660160358, 'Butano': 0}","{'GN': 0, 'Ele': 95.77460372847084, 'Butano': 0}","{'GN': 0, 'Ele': 95.22450124352494, 'Butano': 0}","{'GN': 0, 'Ele': 101.01638545932882, 'Butano': 0}","{'GN': 0, 'Ele': 112.48345269974548, 'Butano': 0}"
Split_rf+PV_no+BC_no,"{'GN': 291.35615879134355, 'Ele': 21.048330124...","{'GN': 291.35615879134355, 'Ele': 20.148228647...","{'GN': 291.35615879134355, 'Ele': 37.744469490...","{'GN': 291.35615879134355, 'Ele': 21.804335327...","{'GN': 291.35615879134355, 'Ele': 91.786028132...","{'GN': 291.35615879134355, 'Ele': 27.075301887...","{'GN': 291.35615879134355, 'Ele': 38.032219320...","{'GN': 291.35615879134355, 'Ele': 22.234638353...","{'GN': 291.35615879134355, 'Ele': 25.625136266...","{'GN': 291.356158791

In [75]:
total_electricity = total_electricity.applymap(sum)
total_electricity

,P211_180_W12_R6,P121_0_W8_R12,P211_270_W10_R12,P211_180_W6_R8,BC0_270,P112_180_W8_R14,P221_270_W10_R6,P221_90_W6_R14,P121_180_W12_R8,P111_0_W8_R6,...,P112_270_W8_R10,P111_270_W10_R14,P221_180_W12_R10,P121_180_W10_R12,P221_270_W10_R10,P121_90_W12_R6,P221_90_W12_R12,P211_180_W8_R12,P121_90_W12_R14,P211_270_W8_R14
Split_rf+PV_yes+BC_yes,42.058383,42.550752,38.874938,43.056119,93.832054,50.248106,38.975079,42.160888,48.536207,43.952437,...,43.223604,42.058346,40.902455,48.290941,38.707234,49.511584,40.925391,41.821486,47.882971,39.002798
Split_rf+PV_yes+BC_no,4.652963,5.145332,1.469518,5.650699,56.426634,12.842686,1.569659,4.755468,11.130787,6.547017,...,5.818184,4.652926,3.497035,10.885522,1.301814,12.106164,3.519971,4.416066,10.477551,1.597378
Split_rf+PV_no+BC_yes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Split_rf+PV_no+BC_no,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Split_nrf+PV_yes+BC_yes,48.274451,49.424594,40.838122,50.605100,169.214511,67.405131,41.072044,48.513896,63.406242,52.698843,...,50.996334,48.274364,45.574275,62.833318,40.446375,65.684663,45.627853,47.721074,61.880324,41.136796
Split_nrf+PV_yes+BC_no,10.869032,12.019174,3.432702,13.199680,131.809091,29.999711,3.666625,11.108477,26.000822,15.293423,...,13.590914,10.868944,8.168855,25.427898,3.040956,28.279243,8.222433,10.315654,24.474904,3.731376
Split_nrf+PV_no+BC_yes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Split_nrf+PV_no+BC_no,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
